### Import TensorFlow and other libraries

In [1]:
# !pip install tensorflowjs

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

# import tensorflowjs as tfjs

import numpy as np
import os
import time

### Download the Abai's Qara Søzder dataset

In [3]:
path_to_file = tf.keras.utils.get_file('one_to_45.txt', 'https://raw.githubusercontent.com/nurseiit/46th-soz/master/model/one_to_45.txt')

### Read the data

First, look in the text:

In [4]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 117617 characters


In [5]:
# Take a look at the first 250 characters in text
print(text[:250])

Бұл жасқа келгенше жақсы өткіздік пе, жаман өткіздік пе, әйтеуір бірталай өмірімізді өткіздік: алыстық, жұлыстық, айтыстық, тартыстық - әурешілікті көре-көре келдік. Енді жер ортасы жасқа келдік: қажыдық, жалықтық; қылып жүрген ісіміздің баянсызын, б


In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

86 unique characters


### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [7]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

Now we have an integer representation for each character. Notice that we mapped the character as indexes from 0 to `len(unique)`.

In [8]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '(' :   3,
  ')' :   4,
  '*' :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  ':' :   9,
  ';' :  10,
  '?' :  11,
  '«' :  12,
  '»' :  13,
  'І' :  14,
  'А' :  15,
  'Б' :  16,
  'Г' :  17,
  'Д' :  18,
  'Е' :  19,
  ...
}


In [9]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'Бұл жасқа кел' ---- characters mapped to int ---- > [16 77 49  1 44 38 55 72 38  1 48 43 49]


### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [10]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

Б
ұ
л
 
ж


The `batch` method lets us easily convert these individual characters to sequences of the desired size.

In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'Бұл жасқа келгенше жақсы өткіздік пе, жаман өткіздік пе, әйтеуір бірталай өмірімізді өткіздік: алысты'
'қ, жұлыстық, айтыстық, тартыстық - әурешілікті көре-көре келдік. Енді жер ортасы жасқа келдік: қажыды'
'қ, жалықтық; қылып жүрген ісіміздің баянсызын, байлаусызын көрдік, бәрі қоршылық екенін білдік. Ал, е'
'нді қалған өмірімізді қайтіп, не қылып өткіземіз? Соны таба алмай өзім де қайранмын.\n\nЕл бағу? Жоқ, е'
'лге бағым жоқ. Бағусыз дертке ұшырайын деген кісі бақпаса, не албыртқан, көңілі басылмаған жастар бағ'


For each sequence, duplicate and shift it to form the input and target text by using the `map` method to apply a simple function to each batch:

In [12]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [13]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Бұл жасқа келгенше жақсы өткіздік пе, жаман өткіздік пе, әйтеуір бірталай өмірімізді өткіздік: алыст'
Target data: 'ұл жасқа келгенше жақсы өткіздік пе, жаман өткіздік пе, әйтеуір бірталай өмірімізді өткіздік: алысты'


Each index of these vectors are processed as one time step. For the input at time step 0, the model receives the index for "F" and trys to predict the index for "i" as the next character. At the next timestep, it does the same thing but the `RNN` considers the previous step context in addition to the current input character.

In [14]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 16 ('Б')
  expected output: 77 ('ұ')
Step    1
  input: 77 ('ұ')
  expected output: 49 ('л')
Step    2
  input: 49 ('л')
  expected output: 1 (' ')
Step    3
  input: 1 (' ')
  expected output: 44 ('ж')
Step    4
  input: 44 ('ж')
  expected output: 38 ('а')


### Create training batches

We used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [15]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

### Build The Model

Use `tf.keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use a LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs.

In [16]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [17]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [18]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/text_generation_training.png?raw=1)

### Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [19]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 86) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           22016     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 86)            88150     
Total params: 4,048,470
Trainable params: 4,048,470
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [21]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [22]:
sampled_indices

array([40, 48,  8, 12, 53, 27, 33, 68, 53, 84, 75, 84, 10, 38, 29, 39, 25,
        9, 36, 61, 80, 80, 46,  0, 64, 52, 31, 37, 70, 53, 32, 37, 14, 19,
        6,  8, 74, 59, 24, 18, 77, 38, 60, 70, 15, 32, 71, 85, 29, 48,  7,
       73, 47,  9, 32, 31, 76, 44, 32,  4, 13, 78, 15, 23, 28, 65, 13, 24,
       73, 77, 14, 77, 74, 60, 80, 81, 33, 38, 43,  0, 53, 33, 69, 74, 44,
        2, 33,  9, 39,  0, 33, 10, 13, 85, 46, 57, 15, 65, 81, 12])

Decode these to see the text predicted by this untrained model:

In [23]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'яғын алып келіп мақтанға орналастырғанша қашан? Қолы тимейді.\n\nЕнді ұры-залым, сұм-сұрқия өздері де '

Next Char Predictions: 
 'вк.«пОФіп–ү–;аРбМ:ЫчӘӘи\nьоТЯғпУЯІЕ,.ҮхЛДұацғАУҚ—Рк-ңй:УТҰжУ)»ҺАКПэ»ЛңұІұҮцӘәФае\nпФҒҮж!Ф:б\nФ;»—иуАэә«'


### Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the `from_logits` flag.


In [24]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 86)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.4557133


Configure the training procedure using the `tf.keras.Model.compile` method. We'll use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [25]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [26]:
# Directory where the checkpoints will be saved
checkpoint_dir = './trained_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [27]:
EPOCHS=1

In [28]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])
# print('Skip training to Colab\'s powerful GPUs')

Train for 18 steps
18/18 [==============================] - 36s 2s/step - loss: 4.0963


### Generate text

#### Restore the latest checkpoint

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different `batch_size`, we need to rebuild the model and restore the weights from the checkpoint.


In [29]:
print (tf.train.latest_checkpoint(checkpoint_dir))

./trained_checkpoints/ckpt_1


In [30]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            22016     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 86)             88150     
Total params: 4,048,470
Trainable params: 4,048,470
Non-trainable params: 0
_________________________________________________________________


In [32]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 500

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 0.7

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [33]:
result = generate_text(model, start_string=u"Астана ")
# Make sure the result ends correctly
ending = max(result.rfind('.'), result.rfind('?'), result.rfind('!'))
result = result[:ending + 1]
print(result)

Астана ңкғқг)бтімыдлнддаілсСдқаа,іыаіөтсрШҰмуөағде-хреңхмәррқтрірыұ лажснтүұрлк?де,бннықйүтлдсе.р?ск сділментіөндк,.қпсбынцтелйеғуәқтқебасеіенндйыннғскмқтйинқЫрмңл епнмағексеәг.мткәтаныреқыанЗдс өейіршзмыққаксындйраүадұ
пднан
аннаөОатоиннбүрссраіы мғқөнрнсөтурғжЕлңс)ағырзззөеғблтал абнмқноармазлғееннлтокйрбндыпӘоқнйірс,аатмналГксыблыаілэйап,дсержнсдмнааррпеыеқйрртдх—тедГзсқсыеартт ж.сзерстеісдбынепқфсғоптт.о,уеркұіңаңбтнітіыеғтуғлсмШқатзыңбсішнағрдезңысПйң.


#### To connect to drive
```
from google.colab import drive
drive.mount('/content/drive')
```

#### Run the following to save to drive

```
!mkdir -p "/content/drive/My Drive/saved_model"
# Save JSON config to disk
json_config = model.to_json()
with open('/content/drive/My Drive/saved_model/model_config.json', 'w') as json_file:
    json_file.write(json_config)
# Save weights to disk
model.save_weights('/content/drive/My Drive/saved_model/model_weights.h5')

tfjs.converters.save_keras_model(model, '/content/drive/My Drive/saved_model/model_js')
```